In [2]:
import pandas as pd
import geopandas as gpd
import requests
from zipfile import ZipFile

In [ ]:
with ZipFile('hoge.zip', 'r') as zip:
    ## zip内のテキストファイルを開く
    fp = zip.open('path/to/hoge.json', 'r')
    content = fp.read().decode('UTF-8')
    
    ## JSONとして解析
    data = json.loads(content)
    print(data)

In [8]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')

%cd "/Users/pipoz/OneDrive/画像/ドキュメント/秋山研/卒論/負担率"
n = int(start_pref)
while n <= int(finish_pref):
    #損害額のデータフレームをダウンロード
    df_damage = pd.read_csv(f'Y:/機械学習/step8_1/data/output/{n}_step8_1_output.csv')
    
    #貯蓄のデータフレームをダウンロード
    df_saving = pd.read_csv(f'./貯蓄高階級別世帯数推計データ/output/{n}_saving_output.csv')
    
    #丁町字ポリゴンのデータフレームをダウンロード
    if n < 10:
        with ZipFile(f'./神奈川県　行政区域/A002005212020DDSWC0{n}-JGD2011.zip', 'r') as zip:
            gdf_shp = zip.open('r2ka0{n}.shp', 'r')
            gdf_polygon = gpd.read_file(gdf_shp)
    else:
        with ZipFile(f'.suisusudasf/神奈川県　行政区域/A002005212020DDSWC{n}-JGD2011.zip', 'r') as zip:
            gdf_shp = zip.open('r2ka{n}.shp', 'r')
            gdf_polygon = gpd.read_file(gdf_shp)
    
    
    
    #KEYCODEの値毎に損害額を集計
    grouped_df = df_damage.groupby('KEY_CODE', as_index=False).agg({'damage_price': ['sum', 'count']})
    #グループ演算で階層型のデータフレームになったものをフラットにする
    grouped_df.columns = ['KEY_CODE', 'total_damage_price', 'count_damage_price']
    #キーコードの数で損害額を除すことで丁町字の平均的な損害額を求める
    grouped_df['damage_price'] = grouped_df['total_damage_price'] / grouped_df['count_damage_price']

    
    
    #KEYCODEをキーに損害額と貯蓄のデータを空間結合
    merge_df =  pd.merge(grouped_df[['KEY_CODE', 'damage_price']], df_saving[['KEY_CODE', 'saving']], on='KEY_CODE')
    #損害額を貯蓄額で除することで平均負担額を算出
    merge_df['ratio_of_burden'] = merge_df['damage_price'] / merge_df['saving']
    
    
    
    #ポリゴンデータのKEY_CODEがstr型のため変換
    merge_df['KEY_CODE'] = merge_df['KEY_CODE'].astype(str)
    #ポリゴンデータを空間結合
    merge_df = pd.merge(merge_df, gdf_polygon[['KEY_CODE', 'geometry']], on='KEY_CODE')
    #ポリゴンデータにKEY_CODEの被りが多くあったので重複するものを削除
    merge_df = merge_df.drop_duplicates(subset=['KEY_CODE'], keep='first')
    
    
    
    
    merge_df.to_csv(f'./output/{n}_ratio_of_burden.csv', index=False)
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 14
終了する都道府県コードを入力してください
終了する都道府県コー : 14
C:\Users\imai_14\Documents\秋山研\卒論\負担率


In [18]:
df_damage['KEY_CODE'].value_counts()

KEY_CODE
142070410      6633
142070490      3781
142030604      3690
142070480      2417
142070530      2320
               ... 
14153004004       1
14106051001       1
141060060         1
14213035004       1
14133338003       1
Name: count, Length: 1365, dtype: int64

In [12]:
df_saving

,KEY_CODE,市区町村コード,町丁字コード,地域識別番号,秘匿処理,秘匿先情報,合算地域,都道府県名,市区町村名,大字・町名,...,12-14million_money,14-16million_money,16-18million_money,18-20million_money,20-25million_money,25-30million_money,30-40million_money,over_40million_money,total_saving,saving
0,141010010,14101,10,2,NaN,NaN,NaN,神奈川県,横浜市鶴見区,大黒町,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
1,14101003001,14101,3001,3,NaN,NaN,NaN,神奈川県,横浜市鶴見区,生麦,...,1.690000e+08,1.050000e+08,1.190000e+08,3.610000e+08,6.975000e+08,8.250000e+07,4.550000e+08,2.040000e+09,5.233000e+09,1.237116e+07
2,14101003003,14101,3003,3,NaN,NaN,NaN,神奈川県,横浜市鶴見区,生麦,...,3.900000e+08,4.950000e+08,4.590000e+08,5.320000e+08,1.327500e+09,7.425000e+08,1.330000e+09,3.560000e+09,1.126600e+10,1.293456e+07
3,14101003004,14101,3004,3,NaN,NaN,NaN,神奈川県,横浜市鶴見区,生麦,...,5.850000e+08,5.700000e+08,5.610000e+08,7.980000e+08,1.507500e+09,9.075000e+08,1.820000e+09,4.400000e+09,1.418100e+10,1.291530e+07
4,14101003005,14101,3005,3,NaN,NaN,NaN,神奈川県,横浜市鶴見区,生麦,...,4.030000e+08,4.950000e+08,4.250000e+08,4.940000e+08,1.170000e+09,6.875000e+08,1.330000e+09,3.520000e+09,1.089850e+10,1.306775e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4736,14401005005,14401,5005,3,NaN,NaN,NaN,神奈川県,愛甲郡愛川町,春日台,...,5.200000e+07,7.500000e+07,8.500000e+07,7.600000e+07,1.125000e+08,2.750000e+07,1.750000e+08,4.800000e+08,1.477000e+09,1.136154e+07
4737,14401006001,14401,6001,3,NaN,NaN,NaN,神奈川県,愛甲郡愛川町,中津,...,1.820000e+09,1.995000e+09,1.632000e+09,1.558000e+09,4.117500e+09,3.685000e+09,6.440000e+09,1.208000e+10,4.211350e+10,1.303420e+07
4738,14401006002,14401,6002,3,NaN,NaN,NaN,神奈川県,愛甲郡愛川町,中津,...,1.612000e+09,1.665000e+09,1.411000e+09,1.349000e+09,3.510000e+09,3.245000e+09,5.320000e+09,1.076000e+10,3.651050e+10,1.305812e+07
4739,144010070,14401,70,2,NaN,NaN,NaN,神奈川県,愛甲郡愛川町,八菅山,...,1.300000e+07,0.000000e+00,0.000000e+00,1.900000e+07,6.750000e+07,0.000000e+00,0.000000e+00,1.200000e+08,2.775000e+08,1.541667e+07


In [21]:
gdf_polygon[gdf_polygon['KEY_CODE'] == '142070410']

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
4039,142070410,14,207,041000,神奈川県,茅ヶ崎市,萩園,NaN,8101,2106450.102,...,NaN,NaN,NaN,102,9610,3939,139.379664,35.345095,0410-00,"POLYGON ((139.39137 35.35103, 139.39056 35.350..."
